In [10]:
#### import global modules
import os
import sys
import pandas as pd
import numpy as np
from pathlib import Path
from yaml import safe_load
import google.oauth2.credentials
from google.cloud import bigquery
import gc

# Set global vars
pth_project = Path(os.getcwd().split('notebooks')[0])
pth_data = pth_project / 'data'
pth_queries = pth_project / 'core' / 'queries'
pth_creds = pth_project / 'conf' / 'local' / 'project_config.yaml'
sys.path.insert(0, str(pth_project))
d_project_config = safe_load(pth_creds.open())
# d_params = safe_load((pth_project / 'core' / 'parameters' / 'common.yaml').open())['data_extract']

# import local modules
from core.utils.gcp import connect_bq_services
# from core.etl.extract import extract_bq_data, extract_pr_codes, format_conv_df, filter_convs

# Connect to google services
bq_client = connect_bq_services(d_project_config['gcp-project-name'])
pd.options.display.max_rows = 100

In [11]:
def extract_bq_data(bq_client, sql=None, pth_query=None):
    if sql is not None:
        df = bq_client.query(sql).to_dataframe()
    elif pth_query is not None:
        sql = pth_query.read_text()
        df = bq_client.query(sql).to_dataframe()
    else:
        raise ValueError('`sql` or `pth_query` should be set')  
    return df

In [12]:
Query='''




DECLARE
  start_dt_snpsht DATE DEFAULT '2022-12-01';
DECLARE
  end_dt_snpsht DATE DEFAULT '2023-02-28';
DECLARE
  churn_dt_snpsht_start DATE DEFAULT '2023-03-01';
DECLARE
  churn_dt_snpsht_end DATE DEFAULT '2023-03-31';  




WITH Customer_Base as (select * from `divgpras-pr-579355.SHS.SHS_churn_model_base_trn_data_dec22_feb23`)

, ADC_BP as ( 
select *,(Best_Practices_1+Best_Practices_2+Best_Practices_3+Best_Practices_4+Best_Practices_5+Best_Practices_6) as Best_Practice_All
from 

(
  -- #1 from_item
  SELECT 
    customer_id,
    best_practices_id,
     case when best_practices_ind=True then 1 else 0 end as Best_practices_flag,

  FROM `cio-datahub-enterprise-pr-183a.src_adc.bq_customer_best_practice`
  where DATE(dt_last_calculate_utc) <= end_dt_snpsht
  QUALIFY ROW_NUMBER() OVER (PARTITION BY customer_id,best_practices_id ORDER BY last_updt_ts DESC) = 1
)
PIVOT
(
  -- #2 aggregate
  sum(Best_practices_flag) AS Best_Practices
  -- #3 pivot_column
  FOR best_practices_id in (1,2,3,4,5,6)
)

)


,ADC_arming AS (

With date_sql as (
select  count(distinct date(date)) as count_of_dates from `cio-datahub-enterprise-pr-183a.src_adc.bq_aggregate_daily_arming_commands`
where date(date)<= end_dt_snpsht and date(date)>= start_dt_snpsht

)

select 
    id_cust as customer_id
    ,sum(count_arm_commands) as sum_arm_commands
    ,sum(count_disarm_commands) as sum_disarm_commands
    ,count (distinct date(date)) as number_days_arming_disarming
    , (select count_of_dates from date_sql ) as count_of_dates_arming
    , round(count (distinct date(date))*100/(select max(count_of_dates) from date_sql),0) as arming_consistency
    , round(sum(count_arm_commands)/(count (distinct date(date))),0) as Avg_arm_per_day
    ,round(sum(count_disarm_commands)/(count (distinct date(date))),0) as Avg_disarm_per_day
from `cio-datahub-enterprise-pr-183a.src_adc.bq_aggregate_daily_arming_commands`
where date(date)<= end_dt_snpsht and date(date)>= start_dt_snpsht
group by customer_id
order by number_days_arming_disarming desc

)


, ADC_login as (

with date_sql as (
select  count(distinct date(login_dt_utc)) as count_of_dates from `cio-datahub-enterprise-pr-183a.src_adc.bq_customer_daily_logins`
where date(login_dt_utc)<= end_dt_snpsht and date(login_dt_utc)>= start_dt_snpsht

)

select customer_id,sum(login_count) as sum_login_count,count (distinct date(login_dt_utc)) as number_of_login_days, (select count_of_dates from date_sql ) as count_of_dates_login, round(count (distinct date(login_dt_utc))*100/(select max(count_of_dates) from date_sql),0) as login_consistency, round(sum(login_count)/(count (distinct date(login_dt_utc))),0) as Avg_login_per_day
from `cio-datahub-enterprise-pr-183a.src_adc.bq_customer_daily_logins`
where date(login_dt_utc)<= end_dt_snpsht and date(login_dt_utc)>= start_dt_snpsht
group by customer_id
order by login_consistency desc



)


, cte_product_mix AS (
  
SELECT distinct cast (ffh_prod.bacct_bus_bacct_num as STRING) AS ban,
         COUNT(DISTINCT CASE WHEN ffh_prod.pi_prod_instnc_typ_cd IN ('SING', 'HSIC', 'TTV', 'SMHM', 'STV', 'DIIC','C') THEN ffh_prod.pi_prod_instnc_typ_cd ELSE NULL END) AS product_mix_all,
         COUNT(DISTINCT CASE WHEN ffh_prod.pi_prod_instnc_typ_cd = 'HSIC' THEN ffh_prod.bus_prod_instnc_id ELSE NULL END) AS hsic_count,
         COUNT(DISTINCT CASE WHEN ffh_prod.pi_prod_instnc_typ_cd = 'SING' THEN ffh_prod.bus_prod_instnc_id ELSE NULL END) AS sing_count,
         COUNT(DISTINCT CASE WHEN ffh_prod.pi_prod_instnc_typ_cd = 'SMHM' THEN ffh_prod.bus_prod_instnc_id ELSE NULL END) AS shs_count,
         COUNT(DISTINCT CASE WHEN ffh_prod.pi_prod_instnc_typ_cd = 'TTV'  THEN ffh_prod.bus_prod_instnc_id ELSE NULL END) AS ttv_count,
         COUNT(DISTINCT CASE WHEN ffh_prod.pi_prod_instnc_typ_cd = 'STV'  THEN ffh_prod.bus_prod_instnc_id ELSE NULL END) AS stv_count,
         COUNT(DISTINCT CASE WHEN ffh_prod.pi_prod_instnc_typ_cd = 'DIIC' THEN ffh_prod.bus_prod_instnc_id ELSE NULL END) AS diic_count   
         FROM `cio-datahub-enterprise-pr-183a.ent_cust_cust.bq_prod_instnc_snpsht` AS ffh_prod
   
   WHERE DATE(ffh_prod.prod_instnc_ts) = end_dt_snpsht
     AND ffh_prod.pi_prod_instnc_stat_cd = 'A' --Active Products
     AND ffh_prod.consldt_cust_typ_cd = 'R' --Regular (not Business)
     AND ffh_prod.pi_prod_instnc_typ_cd IN 
         (
           'DIIC', --Dialup
           'HSIC', --High Speed
           'SING', --Home Phone
           'SMHM', --Smart Home
           'STV',  --Satelite
           'TTV'   --TV
         )
GROUP BY ffh_prod.bacct_bus_bacct_num
--QUALIFY ROW_NUMBER() OVER (PARTITION BY ffh_prod.bacct_bus_bacct_num ORDER BY ffh_prod.prod_instnc_ts ) = 1
)



, ARPU_Calculation as(

with ARPU_full as (
SELECT
  distinct cast (billg_acct_num as string) AS ban,
  EXTRACT(YEAR
  FROM
    bill.bill_dt)||'-'||LPAD(CAST(EXTRACT(MONTH
      FROM
        bill.bill_dt) AS STRING), 2, '0') AS bill_year_month,
  sum(bill.rid_unit_chrg_amt) as ARPU_SMHM,
  sum(tot_inv_amt) as ARPU_FFH
  FROM
  `cio-datahub-enterprise-pr-183a.ent_cust_bill.bq_wln_inv_sum_view` bill
WHERE
  DATE(bill.bill_dt)>=  start_dt_snpsht
  and DATE(bill.bill_dt)<= end_dt_snpsht
  group by ban,bill_year_month
)

select ban, round(avg(ARPU_SMHM),0) as Avg_SMHM_ARPU,round(avg(ARPU_FFH),0) as Avg_FFH_ARPU 
from ARPU_full group by ban

)


,Deact_list as (

select distinct Telus_Cust_ID,SMHM_Churn from 
(select  distinct cast (CUST_ID as string) as Telus_Cust_ID, 1 as SMHM_Churn, PARSE_DATE('%d%h%Y',SUBSTR(ACTIVITY_DATE, 0,9)) as churn_date from `divgpras-pr-579355.SHS.SMHM_Deact_Mar2023`)
where churn_date> churn_dt_snpsht_start and churn_date<=churn_dt_snpsht_end


)


select cb.*,bp.*,arm.* ,lg.*,prod_mix.*,ARPU.*,Deacts.*
from Customer_Base cb
left join ADC_BP bp
on cb.customer_id=bp.customer_id
left join ADC_arming arm
on cb.customer_id=arm.customer_id
left join ADC_login lg
on cb.customer_id=lg.customer_id
left join cte_product_mix as prod_mix
on cb.BAN=prod_mix.ban
left join ARPU_Calculation as ARPU
on cb.BAN=ARPU.ban
left join Deact_list as Deacts
on cb.dealer_customer_id=Deacts.Telus_Cust_ID

'''

In [13]:
DF=extract_bq_data(bq_client, sql=Query)

In [15]:
DF.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 514164 entries, 0 to 514163
Data columns (total 53 columns):
 #   Column                        Non-Null Count   Dtype  
---  ------                        --------------   -----  
 0   Telus_ID                      514164 non-null  object 
 1   BAN                           514164 non-null  object 
 2   contract_start_date           372954 non-null  object 
 3   contract_end_date             514164 non-null  object 
 4   Intial_activation_date        514164 non-null  object 
 5   contract_end_status           514164 non-null  object 
 6   Contract_start_months         372954 non-null  Int64  
 7   dealer_customer_id            404535 non-null  object 
 8   customer_id                   404535 non-null  Int64  
 9   primary_login_id              404535 non-null  Int64  
 10  dealer_name                   514164 non-null  object 
 11  join_date                     404535 non-null  object 
 12  account_type_name             404535 non-nul

In [16]:
DF.head()

,Telus_ID,BAN,contract_start_date,contract_end_date,Intial_activation_date,contract_end_status,Contract_start_months,dealer_customer_id,customer_id,primary_login_id,...,sing_count,shs_count,ttv_count,stv_count,diic_count,ban_2,Avg_SMHM_ARPU,Avg_FFH_ARPU,Telus_Cust_ID,SMHM_Churn
0,98324685,603568717,2011-03-01,2014-03-01,2018-07-20,contract_expired,143,None,<NA>,<NA>,...,0,1,0,0,0,603568717,32.000000000,29.000000000,None,<NA>
1,99616553,603972956,2019-08-02,2022-08-02,2019-08-02,contract_expired,42,None,<NA>,<NA>,...,1,1,1,0,0,603972956,12.000000000,129.000000000,None,<NA>
2,98328849,603589769,2012-08-01,2015-08-01,2018-07-20,contract_expired,126,None,<NA>,<NA>,...,0,1,0,0,0,603589769,30.000000000,32.000000000,None,<NA>
3,98302141,603558561,2018-03-19,2021-03-19,2018-07-12,contract_expired,59,None,<NA>,<NA>,...,0,1,0,0,0,603558561,36.000000000,38.000000000,None,<NA>
4,85154587,602753977,2021-01-07,2024-01-07,2021-01-07,contract_expiring_after_90_days,25,None,<NA>,<NA>,...,1,1,1,0,0,602753977,38.000000000,276.000000000,None,<NA>


In [17]:
DF['SMHM_Churn'].fillna(0,inplace=True)

In [18]:
# DF['churn_date'].fillna(0,inplace=True)

In [19]:
DF['SMHM_Churn'].value_counts()

0    511405
1      2759
Name: SMHM_Churn, dtype: Int64

In [20]:
# DF['churn_date'].value_counts()

In [21]:
# DF['churn_date'].min()

In [22]:
DF['SMHM_Churn'].value_counts(normalize=True)*100

0    99.463401
1     0.536599
Name: SMHM_Churn, dtype: Float64

In [23]:
DF['shs_count'].value_counts()

1     510074
2       3828
3        160
4         14
6          9
25         8
5          7
8          7
18         6
9          6
13         5
20         4
19         4
22         4
17         4
7          4
21         4
11         3
26         2
10         2
12         2
30         1
35         1
16         1
28         1
15         1
24         1
14         1
Name: shs_count, dtype: Int64

In [24]:
DF['BAN'].value_counts()

603568717    1
605664932    1
604840433    1
603642517    1
604841674    1
            ..
604407920    1
605017353    1
605731970    1
227411353    1
605635864    1
Name: BAN, Length: 514164, dtype: int64

In [25]:
DF=DF.drop_duplicates()

In [27]:
DF.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 514164 entries, 0 to 514163
Data columns (total 53 columns):
 #   Column                        Non-Null Count   Dtype  
---  ------                        --------------   -----  
 0   Telus_ID                      514164 non-null  object 
 1   BAN                           514164 non-null  object 
 2   contract_start_date           372954 non-null  object 
 3   contract_end_date             514164 non-null  object 
 4   Intial_activation_date        514164 non-null  object 
 5   contract_end_status           514164 non-null  object 
 6   Contract_start_months         372954 non-null  Int64  
 7   dealer_customer_id            404535 non-null  object 
 8   customer_id                   404535 non-null  Int64  
 9   primary_login_id              404535 non-null  Int64  
 10  dealer_name                   514164 non-null  object 
 11  join_date                     404535 non-null  object 
 12  account_type_name             404535 non-nul

In [230]:
# BAN_list=DF['BAN'].value_counts().rename_axis('BAN').reset_index(name='unique_counts')

In [231]:
# BAN_list_2=BAN_list[BAN_list.unique_counts>1]

In [232]:
# BAN_list_2

,BAN,unique_counts


In [234]:
# DF[DF['BAN']==604371757]
# .to_csv('Data_testing.csv',index=False)

In [28]:
config= bigquery.job.LoadJobConfig()

# config._properties['timePartitioning'] = {'field': 'Month_Year'}
config.write_disposition = bigquery.WriteDisposition.WRITE_TRUNCATE

Table_BQ = 'SHS.SHS_churn_model_train_data_dec22_feb23'

bq_table_instance= bq_client.load_table_from_dataframe(DF, Table_BQ,job_config=config)